# Function File

In [1]:
from pymongo import MongoClient
import numpy as np
import pandas as pd
import time
import re

# Foursquare-required libraries
import os
import requests
from dotenv import load_dotenv

# setup specific to my console, my dataset
client = MongoClient("localhost:27017")
client.list_database_names()

db = client["Ironhack"]
db.list_collection_names()
c = db.get_collection("companies")

## Find All Offices in a City

In [2]:
# Create a dataframe for all offices in the city
def find_offices(city):
    
    # set parameters for list discovery
    filter_ = {"offices.city":city}
    projection = {"_id":0, "name":1, "offices.city":1, "offices.latitude":1, "offices.longitude":1}
    
    # pass all offices into this dict
    name = []
    lat = []
    lon = []

    # loop through each company
    for i in list(c.find(filter_, projection)):    

        # loop through each office in a company
        for y in i['offices']:
            
            # only add values if this is the correct city branch of the company
            if y['city'] == city:
            
                # add mongo values to a dictionary
                name.append(i['name'])
                lat.append(y['latitude'])
                lon.append(y['longitude'])

    # apply new info to a dictionary, then to a dataframe
    to_df = {'name':name, 'lat':lat, 'lon':lon}
    df = pd.DataFrame.from_dict(to_df)
    df.dropna(inplace=True)
    return df

## Add design companies + startups to the previous function: Find All Offices in a City

In [3]:
### This code is not used in the final project

# Create a dataframe for all offices in the city
def find_offices(city):
    
    # set parameters for list discovery
    filter_ = {"offices.city":city}
    projection = {"_id":0, "name":1, "offices.city":1, "offices.latitude":1, "offices.longitude":1, "tag_list":1}
    
    # pass all offices into this dict
    name = []
    lat = []
    lon = []
    tags = []

    # loop through each company
    for i in list(c.find(filter_, projection)):    

        # loop through each office in a company
        for y in i['offices']:
            
            # only add values if this is the correct city branch of the company
            if y['city'] == city:
            
                # add mongo values to a dictionary
                name.append(i['name'])
                lat.append(y['latitude'])
                lon.append(y['longitude'])
                try:
                    if 'design' in i['tag_list']:
                        tags.append(i['tag_list'])
                    elif 'startup' in i['tag_list']:
                        tags.append('startup') # recheck for founded_year less than 5y ago (list from 2013, 
                        # so query companies made just after the '08 crisis)
                    else:
                        tags.append('potential office location')
                except:
                    tags.append('potential office location')

    # apply new info to a dictionary, then to a dataframe
    to_df = {'name':name, 'lat':lat, 'lon':lon, 'tags':tags}
    df = pd.DataFrame.from_dict(to_df)
    df.dropna(inplace=True)
    return df

find_offices('Valencia')


,name,lat,lon,tags
0,e-contratos,39.469301,-0.377068,potential office location
1,Modo Marketing,39.640730,-0.421944,potential office location
2,Efebege,39.450211,-0.392465,potential office location
4,DOMODESK,39.461268,-0.398346,potential office location
5,H2scan,34.442143,-118.598765,potential office location


## Query MongoDB for Designers and return a list

In [4]:
# Create a dataframe for all designers in each city
#def find_offices(city1, city2, city3):
city1, city2, city3 = "Madrid", "Barcelona", "Valencia" # temp line until function definition replaces this

# lists to collect name sand then be turned into a dataframe via a dictionary
design_query_names = []
design_query_tags = []
design_query_cities = []

# set parameters for list discovery
filter_ = {"tag_list":{"$regex":"design"}, "offices.city":city3}
projection = {"_id":0, "name":1, "offices.city":1, "tag_list":1}

# loop through each company to find those which match the criteria
for i in list(c.find(filter_, projection)):

    # only add the designer if this is in one of the cities in the search criteria
    if i['offices']['city'] == city1: #or city2 or city3:

        # add mongo values to a dictionary
        design_query_names.append(i['name'])
        design_query_tags.append(i['tag_list'])
        design_query_cities.append(i['city'])

# apply new info to a dictionary, then to a dataframe
to_df = {'name':design_query_names, 'tags':design_query_tags, 'city':design_query_cities}
df = pd.DataFrame.from_dict(to_df)
#df.dropna(inplace=True)
df
    #return df

,name,tags,city


## Query Foursquare for Starbucks and return a list

In [7]:
# setup search
load_dotenv()
client_id = os.getenv("CLIENT_ID")
client_secret = os.getenv("CLIENT_SECRET")
url = "https://api.foursquare.com/v2/venues/search/"

# input coordinates to find nearest venue
barcelona = [41.385486, 2.170287] # temp coordinates until function defined
query = "Starbucks"

### Remember to limit header queries, or risk reaching your quota!

In [16]:
# input office coordinates and venue to find nearby
query = "starbucks"
origin = [41.385486,2.170287]

def get_fsqr_distance(query, origin):
    url = f"https://api.foursquare.com/v3/places/search?query={query}&ll={origin[0]}%2C{origin[1]}&sort=DISTANCE&limit=1"
    headers = {
        "Accept": "application/json",
        "Authorization": "fsq3nSb08fmNysRWqCGqA/6XjzbUkx+LGcB+JOstG+sNUI8="
    }
    response = requests.get(url, headers=headers)
    return response.json()["results"][0]["geocodes"]
    
nearest_starbucks = get_fsqr_distance(query,origin)
nearest_starbucks

NameError: name 'bcn' is not defined

## Sort through the closest of the returned locations to each office location.

In [ ]:
# pull a city's office locations and design company locations into lists

# for i in office_location:
#   for y in design_companies:
#     if the design company is closer than the last one:
#       store the new value

# probably save to a list bc this will be used for all 9 requirements
# so you will need to pass all 9 lists into a new dict and then df